In [6]:
import pandas as pd
import numpy as np
from pprint import pprint
import pickle

In [2]:
# movie_data = pickle.load( open( "movie_data.pkl", "rb" ) )

In [4]:
test = movie_data[:5]

In [9]:
def parse_weekly_data(weekly_dict):
    '''a function that creates the attributes that are a function 
       of the weekly data (e.g., drop_off_rate) and others. Minimum case yields
       the number of weeks in theatres.
    '''
    pass

In [8]:
def parse_movie_data(movie_dict):
    ''' Takes a dict of a movie scraped from Box Office Mojo and generates
        a row of for each feature to be added to the dataframe.
    '''
    pass

In [7]:
pprint(test)

[{'domestic_total': 77222099,
  'rating': u'PG-13',
  'release_date': datetime.datetime(2010, 6, 11, 0, 0),
  'runtime': 117,
  'title': u'The A-Team',
  'weekly_data': {'avg_gross_by_theatre': ['$10,190',
                                           '$5,875',
                                           '$4,278',
                                           '$2,824',
                                           '$3,235',
                                           '$1,641',
                                           '$1,531',
                                           '$1,533',
                                           '$1,298',
                                           '$1,105',
                                           '$1,040',
                                           '$989',
                                           '$1,406',
                                           '$832'],
                  'change_gross': ['-',
                                   '-42.2%',
                       